In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from pandas.tools.plotting import scatter_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from pprint import pprint
import seaborn as sns
%matplotlib inline

# Question 1: Propensity score matching


In this exercise, you will apply [propensity score matching](http://www.stewartschultz.com/statistics/books/Design%20of%20observational%20studies.pdf), which we discussed in lecture 5 ("Observational studies"), in order to draw conclusions from an observational study.

We will work with a by-now classic dataset from Robert LaLonde's study "[Evaluating the Econometric Evaluations of Training Programs](http://people.hbs.edu/nashraf/LaLonde_1986.pdf)" (1986).
The study investigated the effect of a job training program ("National Supported Work Demonstration") on the real earnings of an individual, a couple of years after completion of the program.
Your task is to determine the effectiveness of the "treatment" represented by the job training program.

#### Dataset description

- `treat`: 1 if the subject participated in the job training program, 0 otherwise
- `age`: the subject's age
- `educ`: years of education
- `race`: categorical variable with three possible values: Black, Hispanic, or White
- `married`: 1 if the subject was married at the time of the training program, 0 otherwise
- `nodegree`: 1 if the subject has earned no school degree, 0 otherwise
- `re74`: real earnings in 1974 (pre-treatment)
- `re75`: real earnings in 1975 (pre-treatment)
- `re78`: real earnings in 1978 (outcome)

If you want to brush up your knowledge on propensity scores and observational studies, we highly recommend Rosenbaum's excellent book on the ["Design of Observational Studies"](http://www.stewartschultz.com/statistics/books/Design%20of%20observational%20studies.pdf). Even just reading the first chapter (18 pages) will help you a lot.

#### 1. A naive analysis

Compare the distribution of the outcome variable (`re78`) between the two groups, using plots and numbers.
To summarize and compare the distributions, you may use the techniques we discussed in lectures 4 ("Read the stats carefully") and 6 ("Data visualization").

What might a naive "researcher" conclude from this superficial analysis?

In [7]:
#load the data
data = pd.read_csv("lalonde.csv")
data.head()

,id,treat,age,educ,black,hispan,married,nodegree,re74,re75,re78
0,NSW1,1,37,11,1,0,1,1,0.0,0.0,9930.0460
1,NSW2,1,22,9,0,1,0,1,0.0,0.0,3595.8940
2,NSW3,1,30,12,1,0,0,0,0.0,0.0,24909.4500
3,NSW4,1,27,11,1,0,0,1,0.0,0.0,7506.1460
4,NSW5,1,33,8,1,0,0,1,0.0,0.0,289.7899


mean (before / after - mean without the zeros )
growth (74-75) (75-78)(75-78 /3)
plot hist of 

In [13]:
data[data['re74'] == 0]

,id,treat,age,educ,black,hispan,married,nodegree,re74,re75,re78
0,NSW1,1,37,11,1,0,1,1,0.0,0.0,9930.0460
1,NSW2,1,22,9,0,1,0,1,0.0,0.0,3595.8940
2,NSW3,1,30,12,1,0,0,0,0.0,0.0,24909.4500
3,NSW4,1,27,11,1,0,0,1,0.0,0.0,7506.1460
4,NSW5,1,33,8,1,0,0,1,0.0,0.0,289.7899
5,NSW6,1,22,9,1,0,0,1,0.0,0.0,4056.4940
6,NSW7,1,23,12,1,0,0,0,0.0,0.0,0.0000
7,NSW8,1,32,11,1,0,0,1,0.0,0.0,8472.1580
8,NSW9,1,22,16,1,0,0,0,0.0,0.0,2164.0220
9,NSW10,1,33,12,0,0,1,0,0.0,0.0,12418.0700


In [10]:
data['re75'].mean()

2184.9382070846914

In [ ]:
bar plot (3 years )

#### 2. A closer look at the data

You're not naive, of course (and even if you are, you've learned certain things in ADA), so you aren't content with a superficial analysis such as the above.
You're aware of the dangers of observational studies, so you take a closer look at the data before jumping to conclusions.

For each feature in the dataset, compare its distribution in the treated group with its distribution in the control group, using plots and numbers.
As above, you may use the techniques we discussed in class for summarizing and comparing the distributions.

What do you observe?
Describe what your observations mean for the conclusions drawn by the naive "researcher" from his superficial analysis.

In [ ]:
boxplot for age and education
plots with the observations 
ratios for the rest 


#### 3. A propensity score model

Use logistic regression to estimate propensity scores for all points in the dataset.
You may use `sklearn` to fit the logistic regression model and apply it to each data point to obtain propensity scores:

```python
from sklearn import linear_model
logistic = linear_model.LogisticRegression()
```

Recall that the propensity score of a data point represents its probability of receiving the treatment, based on its pre-treatment features (in this case, age, education, pre-treatment income, etc.).
To brush up on propensity scores, you may read chapter 3.3 of the above-cited book by Rosenbaum or [this article](https://drive.google.com/file/d/0B4jctQY-uqhzTlpBaTBJRTJFVFE/view).

Note: you do not need a train/test split here. Train and apply the model on the entire dataset. If you're wondering why this is the right thing to do in this situation, recall that the propensity score model is not used in order to make predictions about unseen data. Its sole purpose is to balance the dataset across treatment groups.
(See p. 74 of Rosenbaum's book for an explanation why slight overfitting is even good for propensity scores.
If you want even more information, read [this article](https://drive.google.com/file/d/0B4jctQY-uqhzTlpBaTBJRTJFVFE/view).)

#### 4. Balancing the dataset via matching

Use the propensity scores to match each data point from the treated group with exactly one data point from the control group, while ensuring that each data point from the control group is matched with at most one data point from the treated group.
(Hint: you may explore the `networkx` package in Python for predefined matching functions.)

Your matching should maximize the similarity between matched subjects, as captured by their propensity scores.
In other words, the sum (over all matched pairs) of absolute propensity-score differences between the two matched subjects should be minimized.

After matching, you have as many treated as you have control subjects.
Compare the outcomes (`re78`) between the two groups (treated and control).

Also, compare again the feature-value distributions between the two groups, as you've done in part 2 above, but now only for the matched subjects.
What do you observe?
Are you closer to being able to draw valid conclusions now than you were before?

#### 5. Balancing the groups further

Based on your comparison of feature-value distributions from part 4, are you fully satisfied with your matching?
Would you say your dataset is sufficiently balanced?
If not, in what ways could the "balanced" dataset you have obtained still not allow you to draw valid conclusions?

Improve your matching by explicitly making sure that you match only subjects that have the same value for the problematic feature.
Argue with numbers and plots that the two groups (treated and control) are now better balanced than after part 4.

#### 6. A less naive analysis

Compare the outcomes (`re78`) between treated and control subjects, as you've done in part 1, but now only for the matched dataset you've obtained from part 5.
What do you conclude about the effectiveness of the job training program?

# Question 2: Applied ML

We are going to build a classifier of news to directly assign them to 20 news categories. Note that the pipeline that you will build in this exercise could be of great help during your project if you plan to work with text!

## Part 1

- Load the 20newsgroup dataset. It is, again, a classic dataset that can directly be loaded using sklearn ([link](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html)).  
[TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf), short for term frequencyâ€“inverse document frequency, is of great help when if comes to compute textual features. Indeed, it gives more importance to terms that are more specific to the considered articles (TF) but reduces the importance of terms that are very frequent in the entire corpus (IDF). Compute TF-IDF features for every article using [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). Then, split your dataset into a training, a testing and a validation set (10% for validation and 10% for testing). Each observation should be paired with its corresponding label (the article category).

### Solution

Import libraries:

In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

We retrieve the articles from the sklearn dataset. We can see the 20 categories that we are going to cluster the articles into. We decided to get the data in a random order so that we do not have to randomize the order later when splitting the data into training, testing and validation datasets.

In [3]:
#Fetch data. We download the data if it is not saved already.
newsgroups20 = fetch_20newsgroups(subset='all', shuffle=True, random_state=42, download_if_missing=True)

#20 news categories
categories = newsgroups20.target_names

#printing it!
print ("The 20 categories are:")
pprint(list(newsgroups20.target_names))


The 20 categories are:
['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [4]:
#Function to separate the documents into tokens by splitting on spaces and changing all characters to lowercase.
tokenize = lambda doc: doc.lower().split(" ")

# We create the sklearn TFIDF vectorizer, and compute the tfidf features of the data.
tfidf = TfidfVectorizer(tokenizer=tokenize)
tfidf_results = tfidf.fit_transform(newsgroups20.data)

print(tfidf_results)

  (0, 282387)	0.0140258057605
  (0, 365517)	0.113522139385
  (0, 237867)	0.11127683307
  (0, 450247)	0.110280777821
  (0, 123734)	0.117802347049
  (0, 423435)	0.430992321225
  (0, 268816)	0.147765943013
  (0, 451143)	0.121668965308
  (0, 434140)	0.0520731444266
  (0, 405733)	0.0896450297201
  (0, 197952)	0.0714324612804
  (0, 373078)	0.0757326847264
  (0, 428627)	0.0729556662549
  (0, 417615)	0.0744050019808
  (0, 77954)	0.083588891865
  (0, 431478)	0.136434968273
  (0, 150033)	0.108705621629
  (0, 506414)	0.0461762191086
  (0, 490927)	0.0306860931226
  (0, 173795)	0.13098527592
  (0, 402464)	0.0926395655756
  (0, 160883)	0.0460336953254
  (0, 437567)	0.0542189115102
  (0, 216675)	0.0854213348028
  (0, 137875)	0.0557677685832
  :	:
  (18845, 273981)	0.102179157679
  (18845, 104201)	0.102179157679
  (18845, 94404)	0.102179157679
  (18845, 225732)	0.102179157679
  (18845, 48685)	0.102179157679
  (18845, 31750)	0.102179157679
  (18845, 31749)	0.102179157679
  (18845, 153856)	0.10217915767

Next we separate the data into training, testing and validation data sets. We don't randomize it since we loaded the data in random order earlier.

In [6]:
data_len = len(newsgroups20.data)

data_80 = 8 * int(data_len / 10)
data_90 = 9 * int(data_len / 10)

#80% training data
train_tfidf = tfidf_results[:data_80]
train_data = newsgroups20['target'][:data_80]

#10% testing data
test_tfidf = tfidf_results[data_80:data_90]
test_data = newsgroups20['target'][data_80:data_90]

#10% validation data
val_tfidf = tfidf_results[data_90:]
val_data = newsgroups20['target'][data_90:]

print(train_tfidf.shape, test_tfidf.shape, val_tfidf.shape, train_data.shape, test_data.shape, val_data.shape)


(15072, 591946) (1884, 591946) (1890, 591946) (15072,) (1884,) (1890,)


## Part 2

- Train a random forest on your training set. Try to fine-tune the parameters of your predictor on your validation set using a simple grid search on the number of estimator "n_estimators" and the max depth of the trees "max_depth". Then, display a confusion matrix of your classification pipeline. Lastly, once you assessed your model, inspect the `feature_importances_` attribute of your random forest and discuss the obtained results.

Import libraries:

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV ,StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix

In [8]:
model = RandomForestClassifier()
model.fit(train_tfidf,train_data)
prediction = model.predict(test_tfidf)
score = accuracy_score(prediction,test_data)
score

0.54830148619957542

In [33]:
def testRandomForest(depth, n_estimator):
        random_forest = RandomForestClassifier(max_depth=depth, n_estimators=n_estimator, random_state=0)
        random_forest.fit(train_tfidf,train_data)
        prediction = random_forest.predict(val_tfidf)
        print("depth: "+str(depth)+"\t n_estimators: "+str(n_estimator)+"\t accuracy: "+str(accuracy_score(val_data, prediction)))

In [37]:
depth_init = 30
depth_max = 35
estim_init = 50
estim_max = 200
        
for depth in range(depth_init, depth_max+1,5):
    for n_estimator in range(estim_init, estim_max+1,50):
        testRandomForest(depth, n_estimator)

depth: 30	 n_estimators: 50	 accuracy: 0.704761904762
depth: 30	 n_estimators: 100	 accuracy: 0.733333333333
depth: 30	 n_estimators: 150	 accuracy: 0.739682539683
depth: 30	 n_estimators: 200	 accuracy: 0.751851851852
depth: 35	 n_estimators: 50	 accuracy: 0.703174603175
depth: 35	 n_estimators: 100	 accuracy: 0.753439153439
depth: 35	 n_estimators: 150	 accuracy: 0.759259259259
depth: 35	 n_estimators: 200	 accuracy: 0.772486772487


In [38]:
testRandomForest(40, 1000)

KeyboardInterrupt: 